# Money supply in several countries
Getting data for M0, M1, M2, M3

In [1]:
# standard imports
import numpy as np
import pandas as pd

## 1. The USA
Data available from the St.Louis Federal reserve API. 

The Federal Reserve discontinued the calculation of M3 from 2006.

* [M0](https://fred.stlouisfed.org/series/BOGMBASE) | BOGMBASE
* [M1](https://fred.stlouisfed.org/series/M1NS) | M1NS
* [M2](https://fred.stlouisfed.org/series/M2NS) | M2NS

In [2]:
from full_fred.fred_base import FredBase
from full_fred.fred import Fred
fredB = FredBase()
fred = Fred('../fred_api_key.txt')

In [3]:
fred

In [4]:
m1 = fred.get_series_df('M1NS')

In [9]:
fred_codes = {
    # dict to store the money supply type and its FRED series code 
    # type: code
    'M0': 'BOGMBASE',
    'M1': 'M1NS',
    'M2': 'M2NS'
    }

In [62]:
def get_us_money_supply(code_dict):
    # initiate df
    df = pd.DataFrame(columns=['date', 'value', 'type'])

    for type, code in code_dict.items():
        # drop useless columnms
        cur_df = fred.get_series_df(code).drop(['realtime_start', 'realtime_end'], axis=1)

        # set the type column
        cur_df['type'] = type

        # add to working df
        df = pd.concat([df, cur_df])

    # change value to number
    df['value'] = pd.to_numeric(df['value'])
    
    # M1 and M2 are given in billions, but M0 is given in millions
    # adjust M0 to billions
    df.loc[df['type'] == 'M0', 'value'] /= 1000

    # country column
    df['country'] = 'USA'

    df = df.sort_values(by=['date', 'type']) 
    df = df.reset_index(drop=True)

    return df


In [63]:
usa_money = get_us_money_supply(fred_codes)

In [70]:
usa_money

,date,value,type,country
0,1959-01-01,50.5,M0,USA
1,1959-01-01,142.2,M1,USA
2,1959-01-01,289.8,M2,USA
3,1959-02-01,49.8,M0,USA
4,1959-02-01,139.3,M1,USA
...,...,...,...,...
2302,2022-12-01,19885.2,M1,USA
2303,2022-12-01,21395.4,M2,USA
2304,2023-01-01,5328.4,M0,USA
2305,2023-01-01,19597.0,M1,USA


In [12]:
# check for nan values
usa_money[usa_money.isna().any(axis=1)]

,date,value,type


## 2. Canada
Data Available from the BOC's valet API - documentation [here](https://www.bankofcanada.ca/valet/docs#valet_api). I found the series ID's for the data that I needed by downloading the CSV file on [this page](https://www.bankofcanada.ca/rates/banking-and-financial-statistics/selected-monetary-aggregates-and-their-components-formerly-e1/).

I couldn't find a series for the monetary base so I ended up getting all the data from [Statistics Canada](https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=1010011601). I used their tools to get the series I needed and downloaded it as a csv directly and getting rid of the unneeded stuff they add in manually. When I implement some automation, this will have to change, but it works for now.

In [57]:
can_money_input = pd.read_csv('./csv_files/money_supply_canada.csv')

In [58]:
can_money_input

,Date,M0,M1,M2,M3
0,January 1955,"2,227",..,..,..
1,February 1955,"2,214",..,..,..
2,March 1955,"2,219",..,..,..
3,April 1955,"2,221",..,..,..
4,May 1955,"2,232",..,..,..
...,...,...,...,...,...
811,August 2022,"310,021","1,580,825","2,358,110","3,340,092"
812,September 2022,"319,257","1,580,973","2,374,338","3,371,949"
813,October 2022,"309,182","1,566,375","2,376,916","3,377,242"
814,November 2022,"319,796","1,556,447","2,384,951","3,411,736"


In [59]:
def fix_can_money(df):
    df = df.copy()

    # 1) Date format
    # make the date a datetime column
    df['Date'] = pd.to_datetime(df['Date'])
    # year-month-day format
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

    # 2) Wide format to long format for the values
    df = pd.melt(df, id_vars='Date', value_vars=['M0','M1','M2','M3'])

    # 3) Change column names
    df = df.rename(columns={'Date': 'date', 'variable': 'type'})

    # 4) Value column commas, missing values
    df['value'] = df['value'].str.replace(',','')
    df = df.replace('..', np.NAN)

    # 5) Convert value to billions
    df['value'] = pd.to_numeric(df['value'])
    df['value'] = df['value']/1000

    # 6) Add country column
    df['country'] = 'CAN'


    # 6) re-sort, reindex
    df = df.sort_values(by=['date', 'type']) 
    df = df.reset_index(drop=True)

    return df

In [60]:
can_money = fix_can_money(can_money_input)

In [61]:
can_money

,date,type,value,country
0,1955-01-01,M0,2.227,CAN
1,1955-01-01,M1,NaN,CAN
2,1955-01-01,M2,NaN,CAN
3,1955-01-01,M3,NaN,CAN
4,1955-02-01,M0,2.214,CAN
...,...,...,...,...
3259,2022-11-01,M3,3411.736,CAN
3260,2022-12-01,M0,317.937,CAN
3261,2022-12-01,M1,1549.669,CAN
3262,2022-12-01,M2,2393.203,CAN


## Amalgamate 
Every country finished will be added to the dict and the corresponding country data will get added to the output file.

In [71]:
countries = {
    'USA': usa_money,
    'CAN': can_money
}

In [74]:
def union_countries(country_dict):
    # union
    df = pd.concat([d for d in countries.values()])

    # re-sort, reindex
    df = df.sort_values(by=['date', 'type', 'country']) 
    df = df.reset_index(drop=True)

    return df

In [76]:
money_supply = union_countries(countries)

In [77]:
money_supply.to_csv('csv_files/money_supply.csv')